# Reto emergencias

## Agentes

- Vehicle
  - Can move in highwayArea
- Rescuer
  - Can move, spawns from Vehicle
- Victims
  - Only spawn in building zones
  - Can't move

## Modelos

- CityGrid
  - SingleGrid
  - buildingZoneWidth
  - buildingZoneHeight
  - laneWidth = 3.5
  - lanes = 3
  - highwayWidth = laneWidth * lanes
  - highwayHeight = 1000 
  - crisisType [None, Fire]
  - buildingFloors
  - buildingWidth
  - buildingHeight

A diferencia de otros ejemplos, no queremos realizar una animación basada en una matriz sino en objetos que se mueven libremente en un espacio bidimencional. 

## Imports

In [2]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(metric='euclidian')

ModuleNotFoundError: No module named 'mesa'

In [ ]:
from mesa.model import Model

class Vehicle(Agent):
    def __init__(self, id, model, x, y, width, height):
        super().__init__(id, model)

        self.position = np.array((x,y), dtype=np.float64)

        # position
        vec = (np.random.rand(2) - 0.5) * 10
        
        # initial velocity of agent
        self.velocity = np.array(vec, dtype=np.float64)
        
        # initial acceleration of agent
        vec = (np.random.rand(2) - 0.5) / 2
        self.acceleration = np.array(vec, dtype=np.float64)

        # max turning force
        self.max_force = 0.3

        self.max_speed = 5

        # max distance that bird is looking at
        self.perception = 50

        self.width = width
        self.height = height
    
    def step(self):
        # each step will check the limits of the toroidal space
        self.check_edges()
        # each step will check the neighbors to change velocity and acceleration
        self.check_neighbors()

        self.position = self.position + self.velocity
        self.velocity = self.velocity + self.acceleration

        if np.linalg.norm(self.velocity) > self.max_speed:
            # lower speed in proportion to max_speed
            self.velocity = self.velocity / np.linalg.norm(self.velocity) * self.max_speed

        self.acceleration = np.array((0,0), dtype=np.float64)

    def check_edges(self):
        if self.position.flatten()[0] > self.width:
            self.position[0] = 0
        elif self.position.flatten()[0] < 0:
            self.position[0] = self.width
        
        if self.position.flatten()[1] > self.height:
            self.position[1] = 0
        elif self.position.flatten()[1] < 0:
            self.position[1] = self.height
    
    def check_neighbors(self):
        alignment = self.align()
        cohesion = self.cohesion()
        separation = self.separation()
        self.acceleration = self.acceleration + alignment
        self.acceleration = self.acceleration + cohesion
        self.acceleration = self.acceleration + separation